<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/amazoncustomerreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import dask.dataframe as dd
import gdown

In [2]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#file_id = '1TiwWTY4OT9UCAxOsZCTb9dCmGp34vnDX'
file_id = '1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK'
file_name = 'amazonreviews_train_small.csv'

# Construct the download URL (with confirmation bypass)
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
#gdown.download(url, "amazonreviews_train.csv", quiet=False)
gdown.download(url, file_name, quiet=False)



Downloading...
From (original): https://drive.google.com/uc?export=download&id=1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK
From (redirected): https://drive.google.com/uc?export=download&id=1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK&confirm=t&uuid=f900832a-5fa9-41c7-8688-e455f001ab63
To: /content/amazonreviews_train_small.csv
100%|██████████| 224M/224M [00:01<00:00, 157MB/s]


'amazonreviews_train_small.csv'

In [6]:
import pandas as pd

chunk_size = 100000  # Adjust chunk size as needed
chunks = []  # List to store chunk DataFrames

# Read the CSV in chunks
for chunk in pd.read_csv(file_name, chunksize=chunk_size):
    chunks.append(chunk)  # Store each chunk

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, axis=0)

# Display the first few rows
print(df.head())


   polarity                                              title  \
0         2                     Stuning even for the non-gamer   
1         2              The best soundtrack ever to anything.   
2         2                                           Amazing!   
3         2                               Excellent Soundtrack   
4         2  Remember, Pull Your Jaw Off The Floor After He...   

                                                text  
0  This sound track was beautiful! It paints the ...  
1  I'm reading a lot of reviews saying that this ...  
2  This soundtrack is my favorite music of all ti...  
3  I truly like this soundtrack and I enjoy video...  
4  If you've played the game, you know how divine...  


In [7]:
print(df.tail())


        polarity                                         title  \
499994         1       The book oversimplifies complex matters   
499995         1                           Prepare for offence   
499996         2       This is a great song but BUY THE ALBUM.   
499997         1  What happen to Fram gas filter's Quality ???   
499998         1           NO FILTER ON 2003 Hyundai XG350L!!!   

                                                     text  
499994  I thought this book oversimplified genders too...  
499995  Ridley's conjecture and supposition swirling a...  
499996  Trust me, you'd much rather buy the album than...  
499997  I used this same number of Fram filter before ...  
499998  There is no fuel filter on this year/make. Tru...  


In [8]:
df["polarity"].value_counts()

,count
polarity,
2,252782
1,247217


In [9]:
print(df.shape)

(499999, 3)


In [ ]:
df.describe()

,polarity
count,3600000.0
mean,1.5
std,0.5
min,1.0
25%,1.0
50%,1.5
75%,2.0
max,2.0


In [10]:
df.dtypes

,0
polarity,int64
title,object
text,object


In [11]:

# Basic text cleaning- Remove special characters or puncatuation marks
df['cleaned_reviews'] = df['text'].str.replace('[^a-zA-Z\s]', '', regex=True)

In [12]:
df.head(5)

,polarity,title,text,cleaned_reviews
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...,This sound track was beautiful It paints the s...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,Im reading a lot of reviews saying that this i...
2,2,Amazing!,This soundtrack is my favorite music of all ti...,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine...",If youve played the game you know how divine t...




*   What is the the BERT Tokenizer doing?
*   It converts the cleaned reviews to list because hugging face tokenizers work with lists
* BERT appies word piece tokenization . Converts text into subword tokens and assigns each token a unique id
* padding = True . Ensures that all tokenized sequences in the batch are the same length by adding padding tokens to shorter sequences. This is important because transformers process fixed-length inputs in batches
* truncation = True . Shortens any long reviews that exceed the model's maximum token lime ( e:g 512 for BERT). Ensures that excesskive long reviews dont break the models input size limit
* converts the tokenzied output into a pytorch tensor (torch.tensor) format
* required for training or inference in pytorch-based models
* if using tensorflow , we have to pass th e parameter as return_tensors = "tf"



In [13]:
from transformers import BertTokenizer

# # Initialize a tokenizer from a pretrained model (e.g., BERT)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # Tokenize the reviews
# tokens = tokenizer(df['cleaned_reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
# import torch

# # Save the tokenized output (dictionary of tensors)
# torch.save(tokens, 'amazonreviews_train_small_tokenized_data.pt')



*   Do not remove stop words from the tokens if we are using the dataset to do
sentiment analysis (classifying reviews as positive / negative or neutral)

*  Do not remove stop words from the tokens if we are classifying reviews into various categories (electronics, clothing etc) and may be summarization of the review



In [15]:
# Display tokenized output (first 2 examples)
print(tokens.input_ids[:2])
print(tokens.attention_mask[:2])

tensor([[  101,  2023,  2614,  2650,  2001,  3376,  2009, 23262,  1996, 12411,
          7301,  1999,  2115,  2568,  2061,  2092,  1045,  2052, 28667,  8462,
          4859,  2009,  2130,  2000,  2111,  2040,  5223,  6819,  2094,  2208,
          2189,  1045,  2031,  2209,  1996,  2208, 10381,  4948,  2080,  2892,
          2021,  2041,  1997,  2035,  1997,  1996,  2399,  1045,  2031,  2412,
          2209,  2009,  2038,  1996,  2190,  2189,  2009, 10457,  2185,  2013,
         13587,  9019,  2075,  1998,  3138,  1037,  4840,  2121,  3357,  2007,
         24665,  3686,  7334,  1998,  3969,  3993, 19505,  2009,  2052, 17894,
          3087,  2040, 14977,  2000,  4952,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
import torch

# Load the tokenized data from the .pt file
tokens = torch.load('tokenized_data.pt')

# Extract the tensors
input_ids = loaded_encoding['input_ids']
attention_mask = loaded_encoding['attention_mask']

print(input_ids.shape)         # Should show the shape of input_ids
print(attention_mask.shape)    # Should show the shape of attention_mask


In [17]:

# Extract input IDs and attention mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']
print(input_ids)
print(attention_mask)

tensor([[  101,  2023,  2614,  ...,     0,     0,     0],
        [  101, 10047,  3752,  ...,     0,     0,     0],
        [  101,  2023,  6050,  ...,     0,     0,     0],
        ...,
        [  101,  3404,  2033,  ...,     0,     0,     0],
        [  101,  1045,  2109,  ...,     0,     0,     0],
        [  101,  2045,  2003,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [ ]:
# Set model to evaluation mode (important for inference)
from transformers import BertModel
import torch

model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Get the output of the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Extract the hidden states from the last layer
last_hidden_states = outputs.last_hidden_state

print(last_hidden_states.shape)  # Shape: [batch_size, seq_len, hidden_size]


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]